**Table of Contents**
<div id="toc"></div>

In [100]:
import random
from Bio import Entrez
from Bio import SeqIO
from Bio import GenBank
import collections
import re
import datetime
import dateutil.parser
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import AlignIO
Entrez.email = "aine.otoole@ed.ac.uk"
from ete3 import Tree

# Ref Alignment Production

In [133]:
def sort_out_match_format(k):
    """Regex match (of various formats found on genbank) to genotype, \
    standardize formatting to GPx.xx_Gx.xx or Gx.xx"""
#     print k
    pol = ''
    cap = ''
    for i in ['_','/','-']:
        if i in k and 'P' in k:
            j= k.split(i)
            for item in j:
                if not item.startswith('G'):
                    item = 'G'+item
                if 'P' in item.upper():
                    pol = item.upper().replace('-','.')
                else:
                    cap = item.upper().replace('-','.')
                    
            new_match = "{}_{}".format(pol, cap)
    if not pol and not cap:
        if '.' in k:
            new_match = k.replace('-','_')
        else:
            new_match = k.replace('-','.')
    return new_match

def filter_out_ones_with_no_dates_and_sort(genotypes):
    """only include records that have collection dates associated, \
    return sorted wrt most recent record (by mo and yr)"""
    genotypes_w_dates = {}
    
    #genotypes is a dict with key as genotype 
    #value is a list of tuples of length 3 (record.description, collection.date, accession.id)
    
    for k in genotypes:
        dated = [i for i in genotypes[k] if i[1]!='NA']
        if dated:
            genotypes_w_dates[k]=dated
            
    genotypes_w_fixed_dates = collections.defaultdict(list)
    
    for k in genotypes_w_dates:
        new_dates = []
        for desc, date, acc_id in genotypes_w_dates[k]:
            
            if '/' in date[0]:
                date = date[0].split('/')[0]
            if not type(date)==str:
                date = date[0]
            yourdate = dateutil.parser.parse(date)

            genotypes_w_fixed_dates[k].append((acc_id, str(yourdate).split('-')[:2]))
            
    sorted_by_date = {}
    
    for k in genotypes_w_fixed_dates:
        
        sorted_by_date[k]= [i for i in sorted(genotypes_w_fixed_dates[k], key = lambda x : (int(x[1][0]), int(x[1][1])), reverse=True)]
    
    print len(sorted_by_date)

    return sorted_by_date

def when_regex_works(my_id, my_record, regex, g_dict, s_dict):
    """fix the formatting of the results, \
    add to genotype (defaultdict list) and seq dict (regular dict)"""
    k = sort_out_match_format(regex[0])
    g_dict[k].append((my_record.description, my_record.features[0].qualifiers.get('collection_date','NA'), my_id))
    s_dict[my_id]=my_record.seq
    
def run_regex_search(my_record, on_what_you_ask):
    """run regex for genotype string search on genbank record return the result"""
    genotype_match_str = "[GP]+\d{1,2}[^/]{1}G?P?\w{1,3}\.\d+|G[IV]{1,3}.{1}P?\d+.{1}G?P?\w{1,3}\.\d+|G[IV]{1,3}\.{1}\d{1,2}|G[IV]{1,3}\-{1}\d{1,2}"
    return re.findall(genotype_match_str,on_what_you_ask)
    

    
        

In [135]:
seq_dict = {}
genotypes = collections.defaultdict(list)

for record in SeqIO.parse("refs_from_genbank/all_genbank_norovirus_sequences.gb", "genbank"):
    if len(record)>500:

        org = record.annotations['organism']

        if 'Norovirus' in record.description or 'Norwalk' in record.description:
            rematch = run_regex_search(record, org)
            if rematch:
                when_regex_works(record.id, record, rematch, genotypes, seq_dict)
#                 print record.id
            else:
                rematch = run_regex_search(record, record.description)
                if rematch:
                    when_regex_works(record.id, record, rematch, genotypes, seq_dict)
#                     print record.id

                    
print len(seq_dict)
print len(genotypes)



6413
93


In [106]:
handle1 = Entrez.esearch(db="nucleotide", 
                        term="Norovirus[Organism] AND (biomol_genomic[PROP] AND (500[SLEN]:90000[SLEN])", 
                        idtype="acc",
                        retmax=11000)
records = Entrez.read(handle1)
new_record_ids = []
for record in records["IdList"]:
    if record not in seq_dict:
        new_record_ids.append(record)
print len(new_record_ids)
id_string = ','.join(new_record_ids)
handle2 = Entrez.efetch(db="nucleotide", 
                       id=id_string, 
                       rettype="gb", 
                       retmode="text")
for record in SeqIO.parse(handle2, "genbank"):
    
    if len(record)>500:
        
        if 'Norovirus' in record.description or 'Norwalk' in record.description:
            
            org = record.annotations['organism']
        
            rematch = run_regex_search(record, org)
            
            if rematch:
                when_regex_works(record, rematch, genotypes, seq_dict)
                print record.id, '\n' ,record.description
            
            else:
                rematch = run_regex_search(record,record.description)
                if rematch:
                    when_regex_works(record, rematch, genotypes, seq_dict)
                    print record.id, '\n' ,record.description



3594


In [136]:
my_sorted_genotype_dict_w_dates = filter_out_ones_with_no_dates_and_sort(genotypes)

90


In [146]:
g1, g2, g3, g4, g5, g6 = {},{},{},{},{},{}

dict_list = [g1, g2, g3, g4, g5, g6]
genogroup_list = ['GI.','GII.','GIII.','GIV.','GV.','GVI.']
dict_dict = {}
for i, d in zip(genogroup_list, dict_list):
#     dict_dict[d] = collections.defaultdict(list)
    for k in my_sorted_genotype_dict_w_dates:
        if k.startswith(i):
            dict_dict[d]+=k


# for k in my_sorted_genotype_dict_w_dates:
#     print k, '\t', len(my_sorted_genotype_dict_w_dates[k])
#     for i, d in zip(genogroup_list, dict_list):
#         if k.startswith(i):
#             dict_list[d][k] = my_sorted_genotype_dict_w_dates[k]
# #     for acc_id, date in my_sorted_genotype_dict_w_dates[k]:
# #         print '\t', acc_id, date, len(seq_dict[acc_id])
for i in dict_dict:
    print i
    for j in dict_dict[i]:
        print j

TypeError: unhashable type: 'dict'

# First Level

In [82]:
first_level = {}

for record in SeqIO.parse('refs_from_genbank/nt_seq_reference_genotypes_vinje_orf1.fa','fasta'):
    first_level[record.id]=record.seq
for record in SeqIO.parse('refs_from_genbank/nt_seq_reference_genotypes_vinje_orf2.fa','fasta'):
    first_level[record.id]=record.seq
    
print first_level.keys()

['GIV.1', 'GII.4_New_Orleans', 'GIV.2', 'GII.4_NSW001P', 'GII.12', 'GII.6a', 'GII.6c', 'GII.6b', 'GI.5a', 'GI.5b', 'GII.P22', 'GII.P20', 'GII.P21', 'GII.P7', 'GII.P6', 'GII.P5', 'GV.2', 'GII.P3', 'GII.P2', 'GII.P1', 'GII.P8', 'GIII.P3', 'GIII.P2', 'GIII.P1', 'GVII.', 'GII.4_Apeldoorn', 'GII.3b', 'GII.3c', 'GII.3a', 'GVII.P', 'GII.Pg', 'GII.Pf', 'GII.Pe', 'GII.Pc', 'GII.Pa', 'GII.11', 'GII.Pn', 'GII.Pm', 'GII.Pk', 'GII.Pj', 'GII.Ph', 'GIII.2', 'GIII.3', 'GIII.1', 'GI.3b', 'GI.3c', 'GI.3a', 'GI.3d', 'GV.1', 'GIV.P2', 'GI.8', 'GI.9', 'GI.2', 'GI.1', 'GI.4', 'GI.6a', 'GI.6b', 'GII.21', 'GII.20', 'GII.22', 'GII.18', 'GII.19', 'GII.10', 'GVI.P1', 'GVI.P2', 'GII.13', 'GII.14', 'GII.15', 'GII.16', 'GII.17', 'GII.8', 'GII.9', 'GI.Pf', 'GII.4', 'GI.Pd', 'GI.Pc', 'GI.Pb', 'GI.Pa', 'GII.1', 'GII.5', 'GII.2', 'GII.7', 'GII.P16', 'GII.P15', 'GII.P13', 'GII.P12', 'GII.P11', 'GII.P18', 'GI.P9', 'GI.P8', 'GI.P7', 'GI.P6', 'GI.P5', 'GI.P4', 'GI.P3', 'GI.P2', 'GI.P1', 'GV.P2', 'GV.P1', 'GI.7b', 'GI.7c', 

In [ ]:
#for each read align to a suite prototypic references using minimap2

In [84]:
%%bash

for READ in SEQFILE; do
    minimap2 

Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer (preferrable for PacBio)
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM       bandwidth used in chaining and DP-based alignment [500]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all mode)
    -p

# Second Level

In [ ]:
# Dynamically add confidence as more reads are assigned to a particular genotype
# more confidence if from one barcode the reads are being assigned to a single genotype in a particular location 
# less confidence if there is variance in best alignments

# De novo assembly

Using spades? 